In [1]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 133 kB 5.1 MB/s 


In [2]:
import tensorflow as tf
import keras_tuner as kt
print(tf.__version__)   # >2.8.0
print(kt.__version__)   # >1.1.2

import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
from tensorflow import keras 
from tensorflow.keras import layers
import keras_tuner as kt
import numpy as np
import IPython

2.8.0
1.1.2


In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings, random
warnings.filterwarnings(action='ignore')

from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import StratifiedKFold

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
train_df = pd.read_csv('/content/drive/MyDrive/Likelion_Semi_project_03/data/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Likelion_Semi_project_03/data/test.csv')

# Preprocessing

In [6]:
def preprocessing(df):
    df = df.drop(['index','FLAG_MOBIL'], axis = 1)
    
    df.gender = df.gender.replace({'F' : 0, 'M' : 1})
    df.car = df.car.replace({'N' : 0, 'Y' : 1})
    df.reality = df.reality.replace({'N' : 0, 'Y' : 1})
    df.occyp_type = df.occyp_type.fillna('NaN')
    
    df['age'] = df.DAYS_BIRTH.apply(lambda x : -x // 365)
    df.DAYS_EMPLOYED = (-1) * df.DAYS_EMPLOYED 
    df.loc[(df.DAYS_EMPLOYED < 0), 'DAYS_EMPLOYED'] = 0
    df.begin_month = (-1) * df.begin_month
    
    df = df.drop(['credit','DAYS_BIRTH'], axis = 1)
    
    return df

In [7]:
X = preprocessing(train_df)
y = train_df.credit
# X_test = preprocessing(X_test)
X

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_EMPLOYED,work_phone,phone,email,occyp_type,family_size,begin_month,age
0,0,0,0,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,4709,0,0,0,NaN,2.0,6.0,38
1,0,0,1,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,1540,0,0,1,Laborers,3.0,5.0,31
2,1,1,1,0,450000.0,Working,Higher education,Married,House / apartment,4434,0,1,0,Managers,2.0,22.0,52
3,0,0,1,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,2092,0,1,0,Sales staff,2.0,37.0,41
4,0,1,1,0,157500.0,State servant,Higher education,Married,House / apartment,2105,0,0,0,Managers,2.0,26.0,41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,0,0,0,2,225000.0,State servant,Secondary / secondary special,Married,House / apartment,1984,0,0,0,Core staff,4.0,2.0,33
26453,0,0,1,1,180000.0,Working,Higher education,Separated,House / apartment,2475,0,0,0,NaN,2.0,47.0,41
26454,0,1,0,0,292500.0,Working,Secondary / secondary special,Civil marriage,With parents,2015,0,0,0,Core staff,2.0,25.0,27
26455,1,0,1,0,171000.0,Working,Incomplete higher,Single / not married,House / apartment,107,0,0,0,Laborers,1.0,59.0,27


# Validation set 만들기

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2)

In [9]:
numeric_features = ['child_num', 'income_total', 'DAYS_EMPLOYED', 'family_size', 'begin_month', 'age']
numeric_transformer = StandardScaler()

categorical_features = ['income_type', 'edu_type', 'family_type', 'house_type','occyp_type']
categorical_transformer = OneHotEncoder(categories='auto', handle_unknown = 'ignore')

# label_features = ['credit']
# label_transformer = LabelEncoder()

preprocessor = ColumnTransformer(
                transformers=[
                    ('num', numeric_transformer, numeric_features),
                    ('cat', categorical_transformer, categorical_features)
                ], remainder='passthrough'
                )

In [10]:
preprocessor.fit(X_train)
scaled_X_train = preprocessor.transform(X_train)
scaled_X_val = preprocessor.transform(X_val)

In [11]:
pd.DataFrame(scaled_X_train)

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,51
0,0.753720,-0.736740,-0.828022,0.869478,1.620163,0.243021,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
1,-0.569259,-0.604355,-0.388568,-1.293868,-0.555600,1.374005,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
2,0.753720,0.807746,-0.189969,0.869478,-1.522606,-1.148959,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,-0.569259,0.013439,0.239766,-0.212195,0.894909,-0.017975,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,-0.569259,-0.516099,1.027825,-0.212195,-0.434724,-0.452969,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21160,-0.569259,1.910949,-0.632803,-0.212195,-0.072097,-1.235958,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
21161,-0.569259,-0.869124,0.241457,-0.212195,1.015784,0.852012,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
21162,-0.569259,-0.648483,-0.255464,-1.293868,-0.857789,0.852012,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
21163,2.076700,0.366464,-0.691115,1.951151,-0.616038,-0.539968,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0


# Bayesian HPO

In [12]:
import tensorflow as tf
from tensorflow.keras import datasets, utils
from tensorflow.keras import models, layers, activations, initializers, losses, optimizers, metrics

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import model_selection, preprocessing

In [13]:
scaled_X_train.shape

(21165, 52)

In [14]:
y_train = utils.to_categorical(y_train)
y_val = utils.to_categorical(y_val)
y_train

array([[0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.]], dtype=float32)

In [15]:
y_train.shape

(21165, 3)

### Hyper_model

In [16]:
# Build the hyper-model
def build_hyper_model(hp):
    
    model = keras.Sequential()
    model.add(layers.Flatten())
        
    ### Hidden layer 만들기 (Search Space)
    # Tune the number of hidden layer (Choose an optimal value between 1~3)
    for layer_num in range(hp.Int('num_layers', min_value=1, max_value=3)):   # hp.int() : 후보 만드는 함수
        # Tune the number of perceptrons in a dense layer (Choose an optimal value between 32~512) 
        hp_units = hp.Int('units_' + str(layer_num), min_value=32, max_value=512, step=32) # 32:512 & step 32, all parameter names should be unique (we name the inner parameters 'units_' + str(i))  & 32, 512 포함
        hp_activations = hp.Choice('activation_' + str(layer_num), values=['relu', 'elu']) # hp.Choice : 문자열, 숫자열로 되있는 후보 중 하나 골라주는 것
        model.add(layers.Dense(units = hp_units, activation = hp_activations))

    # Output layer
    model.add(layers.Dense(3, activation='softmax')) # class 10 : 0~9
    hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 
    
    model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.CategoricalCrossentropy(),
                metrics = ['accuracy'])
    
    return model

In [17]:
# Select tuner and compile it
tuner = kt.BayesianOptimization(build_hyper_model,
                                objective = 'val_loss', # Hyper-params tuning을 위한 목적함수 설정 (metric to minimize or maximize)
                                max_trials = 10, # 서로 다른 Hyper-params 조합으로 시도할 총 Trial 횟수 설정
                                directory = 'test_prac_dir', # Path to the working directory
                                project_name = 'hyper_2') # Name to use as directory name for files saved by this Tuner
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
activation_0 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'elu'], 'ordered': False}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


### Train

In [18]:
# Train the model
tuner.search(scaled_X_train, y_train, epochs=100, validation_data = (scaled_X_val, y_val))

Trial 10 Complete [00h 04m 22s]
val_loss: 0.827477216720581

Best val_loss So Far: 0.8114632964134216
Total elapsed time: 00h 41m 18s


In [19]:
tuner.results_summary(num_trials=3) 

Results summary
Results in test_prac_dir/hyper_2
Showing 3 best trials
Trial summary
Hyperparameters:
num_layers: 2
units_0: 288
activation_0: relu
learning_rate: 0.0001
units_1: 320
activation_1: relu
units_2: 416
activation_2: relu
Score: 0.8114632964134216
Trial summary
Hyperparameters:
num_layers: 2
units_0: 480
activation_0: relu
learning_rate: 0.0001
units_1: 224
activation_1: relu
units_2: 224
activation_2: relu
Score: 0.8117554187774658
Trial summary
Hyperparameters:
num_layers: 2
units_0: 192
activation_0: relu
learning_rate: 0.0001
units_1: 288
activation_1: relu
units_2: 512
activation_2: relu
Score: 0.8198148012161255


In [20]:
# Check the result 
top3_models = tuner.get_best_hyperparameters(num_trials=3)
for idx, model in enumerate(top3_models):
    print('Model performance rank :', idx)
    print(model.values)
    print()

Model performance rank : 0
{'num_layers': 2, 'units_0': 288, 'activation_0': 'relu', 'learning_rate': 0.0001, 'units_1': 320, 'activation_1': 'relu', 'units_2': 416, 'activation_2': 'relu'}

Model performance rank : 1
{'num_layers': 2, 'units_0': 480, 'activation_0': 'relu', 'learning_rate': 0.0001, 'units_1': 224, 'activation_1': 'relu', 'units_2': 224, 'activation_2': 'relu'}

Model performance rank : 2
{'num_layers': 2, 'units_0': 192, 'activation_0': 'relu', 'learning_rate': 0.0001, 'units_1': 288, 'activation_1': 'relu', 'units_2': 512, 'activation_2': 'relu'}



In [21]:
best_hps = top3_models[0]

print("""
The hyperparameter search is complete. 
* Optimal # of layers : {}
* Optimal value of the learning-rate : {}""".format(best_hps.get('num_layers'), best_hps.get('learning_rate')))

for layer_num in range(best_hps.get('num_layers')):
    print('Layer {} - # of Perceptrons :'.format(layer_num), best_hps.get('units_' + str(layer_num)))
    print('Layer {} - Applied activation function :'.format(layer_num), best_hps.get('activation_' + str(layer_num)))


The hyperparameter search is complete. 
* Optimal # of layers : 2
* Optimal value of the learning-rate : 0.0001
Layer 0 - # of Perceptrons : 288
Layer 0 - Applied activation function : relu
Layer 1 - # of Perceptrons : 320
Layer 1 - Applied activation function : relu


### Test 예측

In [22]:
# Get the best model from trials
models = tuner.get_best_models(num_models=3) # Keras Sequential models
# 10개의 trials 중 가장 좋은 model 꺼내줌
models

In [23]:
top_model = models[0]
#top_model.summary()
top_model

In [24]:
scaled_X_val.shape, y_val.shape

((5292, 52), (5292, 3))

In [25]:
y_pred = top_model.predict(scaled_X_val)
logloss = log_loss(y_val, y_pred)
print(logloss)

0.8114635594016887


In [26]:
for trial in tuner.oracle.get_best_trials(num_trials=3):
    print('Trial-score is :', trial.score)
    print('Trial-directory(trial_id) is :', trial.trial_id)
    print()

Trial-score is : 0.8114632964134216
Trial-directory(trial_id) is : 01

Trial-score is : 0.8117554187774658
Trial-directory(trial_id) is : 08

Trial-score is : 0.8198148012161255
Trial-directory(trial_id) is : 07



In [27]:
top_model.evaluate(scaled_X_val, y_val)

166/166 [==============================] - 1s 2ms/step - loss: 0.8115 - accuracy: 0.6869


[0.8114632964134216, 0.6868858933448792]

In [28]:
results = top_model.evaluate(scaled_X_val, y_val)
print('Cross-entropy :', results[0])
print('Accuracy :', results[1])

166/166 [==============================] - 0s 2ms/step - loss: 0.8115 - accuracy: 0.6869
Cross-entropy : 0.8114632964134216
Accuracy : 0.6868858933448792
